In [1]:
import pandas as pd

from nltk.tokenize import word_tokenize
from string import punctuation
import nltk
from unicodedata import normalize
from nltk.stem import RSLPStemmer
from nltk.util import ngrams
from nltk.tokenize import RegexpTokenizer

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('rslp')

import time

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Flavio\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Flavio\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package rslp to
[nltk_data]     C:\Users\Flavio\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!


In [2]:
dados = pd.read_csv("base_20230428_douglas.csv")

In [3]:
colId = range(len(dados))

In [4]:
dados

,codProposicao,txtSiglaTipo,numAno,numNumero,txtNome,txtEmenta,txtExplicacaoEmenta,txtIndexacao,txtInteiroTeor
0,16357,PL,1999,1165,PL 1165/1999,"Altera dispositivo da Lei nº 8.987, de 13 de f...",Estabelece que as concessionárias disponibiliz...,"Alteração, Lei das Concessões de Serviços Públ...",Ofício nº 1416 (SF) ...
1,19098,PL,1992,3097,PL 3097/1992,Dispõe sobre a eleição de diretores de fundos ...,NaN,"NORMAS, ELEIÇÃO DIRETA, EMPREGADO, APOSENTADO,...",COMISSÃO DE CONSTITUIÇÃO E JUSTIÇA E DE REDAÇÃ...
2,20464,PL,2000,3927,PL 3927/2000,Altera a composição dos Tribunais Regionais do...,"Altera a composição do TRT da 5ª região, 6ª re...","Alteração, Lei Federal, composição, Tribunal R...","COMISSÃO DE TRABALHO, DE ADMINISTRAÇÃO E SERVI..."
3,20683,PL,1998,4117,PL 4117/1998,Dispõe sobre o acesso a ambientes de uso colet...,NaN,"Autorização, pessoa portadora de deficiência, ...",Câmara dos Deputados\n Departamento ...
4,20857,PL,1998,4395,PL 4395/1998,Estabelece as Diretrizes Nacionais de Defesa C...,NaN,"Fixação, diretrizes, defesa civil, (Sindec), d...",- 1 - \nCOMISSÃO DE CONSTITUIÇÃO E JUSTIÇA E D...
...,...,...,...,...,...,...,...,...,...
57104,2358873,PL,2023,2233,PL 2233/2023,Altera o parágrafo único do art. 71 do Código ...,NaN,NaN,1 \n \n \n \n \nCÂMARA DOS DEPUTADOS \nPROJET...
57105,2358874,PL,2019,3616,PL 3616/2019,"Altera a Lei nº 9.503, de 23 de setembro de 19...",NaN,NaN,"Altera a Lei nº 9.503, de 23 de setembro\nde 1..."
57106,2358875,PL,2019,1822,PL 1822/2019,"Altera a Lei nº 11.340, de 7 de agosto de 2006...",NaN,NaN,"Altera a Lei nº 11.340, de 7 de agosto de\n200..."
57107,2358877,PL,2019,3815,PL 3815/2019,"Altera a Lei nº 7.565, de 19 de dezembro de 19...",NaN,NaN,"Altera a Lei nº 7.565, de 19 de dezembro\nde ..."


## Sem pré processamento

In [9]:
dados = dados[["txtNome","txtInteiroTeor"]]
dados1 = dados.assign(id=colId)

In [ ]:
dados1

In [10]:
dados1 = dados1.rename(columns={'txtInteiroTeor': 'contents'})

In [11]:
dados1.to_json("sem_pre_processamento/sem_pre_processamento.json",orient="records")

## Letra minuscula

In [12]:
def preprocess(txt):
    txt = str(txt)
    terms = word_tokenize(txt.lower())
    terms = " ".join(terms)
    return terms

In [9]:
dados1["txtInteiroTeor"] = dados1["txtInteiroTeor"].map(preprocess)

In [10]:
dados1 = dados1.rename(columns={"txtInteiroTeor":"contents"})

In [11]:
dados1.to_json("lower/pre_processamento_lower.json",orient="records")

## Letra mínuscula + remoção de pontuação

In [12]:
dados = pd.read_csv("proposicao-tema-completo.csv")

In [13]:
colId = range(len(dados))

In [14]:
dados = dados[["txtNome","imgArquivoTeorPDF"]]
dados1 = dados.assign(id=colId)

In [15]:
def preprocess(txt):
    txt = str(txt)
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = " ".join(terms)
    return terms

In [16]:
dados1["imgArquivoTeorPDF"] = dados1["imgArquivoTeorPDF"].map(preprocess)

In [17]:
dados1.to_json("lower_remocao_punctuation/pre_processamento_lower_remocao_punctuation.json",orient="records")

## Letra mínuscula + remoção de pontuação e remoção de acentuação

In [18]:
dados = pd.read_csv("proposicao-tema-completo.csv")

In [19]:
colId = range(len(dados))

In [20]:
dados = dados[["txtNome","imgArquivoTeorPDF"]]
dados1 = dados.assign(id=colId)

In [21]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = str(txt)
    txt = _remove_acentos(txt)
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = " ".join(terms)
    return terms

In [ ]:
a = time.time()

In [22]:
dados1["imgArquivoTeorPDF"] = dados1["imgArquivoTeorPDF"].map(preprocess)

In [ ]:
d = time.time()

In [23]:
dados1.to_json("lower_remocao_punctuation_acentuacao/pre_processamento_lower_remocao_punctuation_acentuacao.json",orient="records")

## Letra mínuscula + remoção de pontuação + remoção de acentuação e remoção de stopword

In [5]:
dados = pd.read_csv("base_20230428_douglas.csv")

In [6]:
colId = range(len(dados))

In [7]:
dados = dados[["txtNome","txtInteiroTeor"]]
dados1 = dados.assign(id=colId)

In [8]:
dados1 = dados1.rename(columns={"txtInteiroTeor":"contents"})

In [9]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = str(txt)
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [word for word in terms if word not in stopwords]
    terms = " ".join(terms)
    return terms

In [10]:
a = time.time()

In [11]:
dados1["contents"] = dados1["contents"].map(preprocess)

In [12]:
d = time.time()

In [13]:
print("Duração: %f" %(d-a))

Duração: 142.550391


In [19]:
dados1.to_json("lower_remocao_punctuation_acentuacao_stopwords/pre_processamento_lower_remocao_punctuation_acentuacao_stopwords.json",orient="records")

# Stemming

## Stemming (RSLP)

In [30]:
dados = pd.read_csv("base_20230428_douglas.csv")

In [31]:
colId = range(len(dados))

In [32]:
dados = dados[["txtNome","txtInteiroTeor"]]
dados1 = dados.assign(id=colId)

In [33]:
def preprocess(txt):
    txt = str(txt)
    stemmer = RSLPStemmer()
    terms = word_tokenize(txt)
    terms = [stemmer.stem(word) for word in terms]
    terms = " ".join(terms)
    return terms

In [34]:
dados1["txtInteiroTeor"] = dados1["txtInteiroTeor"].map(preprocess)

In [35]:
dados1.to_json("rslp/pre_processamento_rslp.json",orient="records")

## Stemming (Savoy)

In [36]:
dados = pd.read_csv("base_20230428_douglas.csv")

In [37]:
colId = range(len(dados))

In [38]:
dados = dados[["txtNome","txtInteiroTeor"]]
dados1 = dados.assign(id=colId)

In [28]:
class Savoy:

    def __removeAllPTAccent(self, old_word):
        word = list(old_word)
        len_word = len(word)-1
        for i in range(len_word, -1, -1):
            if word[i] == 'ä':
                word[i] = 'a'
            if word[i] == 'â':
                word[i] = 'a'
            if word[i] == 'à':
                word[i] = 'a'
            if word[i] == 'á':
                word[i] = 'a'
            if word[i] == 'ã':
                word[i] = 'a'
            if word[i] == 'ê':
                word[i] = 'e'
            if word[i] == 'é':
                word[i] = 'e'
            if word[i] == 'è':
                word[i] = 'e'
            if word[i] == 'ë':
                word[i] = 'e'
            if word[i] == 'ï':
                word[i] = 'i'
            if word[i] == 'î':
                word[i] = 'i'
            if word[i] == 'ì':
                word[i] = 'i'
            if word[i] == 'í':
                word[i] = 'i'
            if word[i] == 'ü':
                word[i] = 'u'
            if word[i] == 'ú':
                word[i] = 'u'
            if word[i] == 'ù':
                word[i] = 'u'
            if word[i] == 'û':
                word[i] = 'u'
            if word[i] == 'ô':
                word[i] = 'o'
            if word[i] == 'ö':
                word[i] = 'o'
            if word[i] == 'ó':
                word[i] = 'o'
            if word[i] == 'ò':
                word[i] = 'o'
            if word[i] == 'õ':
                word[i] = 'o'
            if word[i] == 'ç':
                word[i] = 'c'

        new_word = "".join(word)
        return new_word

    def __finalVowelPortuguese(self, word):
        len_word = len(word)
        if len_word > 3:
            if word[-1] == 'e' or word[-1] == 'a' or word[-1] == 'o':
                word = word[:-1]

        return word

    def __remove_PTsuffix(self, word):
        len_word = len(word)

        if len_word > 3:
            if word[-1] == 's' and word[-2] == 'e' and (word[-3] == 'r' or word[-3] == 's' or word[-3] == 'z' or word[-3] == 'l'):
                word = word[:-2]
                return word
        if len_word > 2:
            if word[-1] == 's' and word[-2] == 'n':
                new_word = list(word)
                new_word[-2] = 'm'
                sing = "".join(new_word)
                sing = sing[:-1]
                return sing

        if len_word > 3:
            if (word[-1] == 's' and word[-2] == 'i') and (word[-3] == 'e' or word[-3] == 'é'):
                new_word = list(word)
                new_word[-3] = 'e'
                new_word[-2] = 'l'
                sing = "".join(new_word)
                sing = sing[:-1]
                return sing

        if len_word > 3:
            if word[-1] == 's' and word[-2] == 'i' and word[-3] == 'a':
                new_word = list(word)
                new_word[-2] = 'l'
                sing = "".join(new_word)
                sing = sing[:-1]
                return sing

        if len_word > 3:
            if word[-1] == 's' and word[-2] == 'i' and word[-3] == 'ó':
                new_word = list(word)
                new_word[-3] = 'o'
                new_word[-2] = 'l'
                sing = "".join(new_word)
                sing = sing[:-1]
                return sing

        if len_word > 3:
            if word[-1] == 's' and word[-2] == 'i':
                new_word = list(word)
                new_word[-1] = 'l'
                sing = "".join(new_word)
                return sing

        if len_word > 2:
            if word[-1] == 's' and word[-2] == 'e' and word[-3] == 'õ':
                new_word = list(word)
                new_word[-3] = 'ã'
                new_word[-2] = 'o'
                sing = "".join(new_word)
                sing = sing[:-1]
                return sing
            if word[-1] == 's' and word[-2] == 'e' and word[-3] == 'ã':
                new_word = list(word)
                new_word[-2] = 'o'
                sing = "".join(new_word)
                sing = sing[:-1]
                return sing

        if len_word > 5:
            if word[-1] == 'e' and word[-2] == 't' and word[-3] == 'n' and word[-4] == 'e' and word[-5] == 'm':
                word = word[:-5]
                return word

        if len_word > 2:
            if word[-1] == 's':
                word = word[:-1]

        return word

    def __normFemininPortuguese(self, word):

        len_word = len(word)

        if len_word < 3 or word[-1] != 'a':
            return word

        if len_word > 6:

            if word[-2] == 'h' and word[-3] == 'n' and word[-4] == 'i':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

            if word[-2] == 'c' and word[-3] == 'a' and word[-4] == 'i':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

            if word[-2] == 'r' and word[-3] == 'i' and word[-4] == 'e':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

        if len_word > 5:
            if word[-2] == 'n' and word[-3] == 'o':
                new_word = list(word)
                new_word[-3] = 'ã'
                new_word[-2] = 'o'
                masc = "".join(new_word)
                masc = masc[:-1]
                return masc

            if word[-2] == 'r' and word[-3] == 'o':
                word = word[:-1]
                return word

            if word[-2] == 's' and word[-3] == 'o':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

            if word[-2] == 's' and word[-3] == 'e':
                new_word = list(word)
                new_word[-3] = 'ê'
                masc = "".join(new_word)
                masc = masc[:-1]
                return masc

            if word[-2] == 'c' and word[-3] == 'i':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

            if word[-2] == 'd' and word[-3] == 'i':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

            if word[-2] == 'd' and word[-3] == 'a':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

            if word[-2] == 'v' and word[-3] == 'i':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

            if word[-2] == 'm' and word[-3] == 'a':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

            if word[-2] == 'n':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

        return word

    def stem(self, word):
        len_word = len(word)
        if len_word > 2:
            word = self.__remove_PTsuffix(word)
            word = self.__normFemininPortuguese(word)
            word = self.__finalVowelPortuguese(word)
            word = self.__removeAllPTAccent(word)

        return word


In [40]:
def preprocess(txt):
    txt = str(txt)
    stemmer = Savoy()
    terms = word_tokenize(txt)
    terms = [stemmer.stem(word) for word in terms]
    terms = " ".join(terms)
    return terms

In [41]:
dados1["txtInteiroTeor"] = dados1["txtInteiroTeor"].map(preprocess)

In [42]:
dados1.to_json("savoy/pre_processamento_savoy.json",orient="records")

## Stemming (RSLP-S)

In [43]:
dados = pd.read_csv("base_20230428_douglas.csv")

In [44]:
colId = range(len(dados))

In [45]:
dados = dados[["txtNome","txtInteiroTeor"]]
dados1 = dados.assign(id=colId)

In [29]:
class RSLP_S:
    def __plural_reduction(self, word):
        excep = ["lápis","cais","mais","crúcis","biquínis","pois","depois","dois","leis" ]
        excep_s = ["aliás","pires","lápis","cais","mais","mas","menos", "férias","fezes","pêsames","crúcis","gás", "atrás","moisés","através","convés","ês", "país","após","ambas","ambos","messias"]

        len_word = len(word)
        new_word = list(word)

        if len_word >= 3:
            if new_word[-1] == 's' and new_word[-2] == 'n':
                new_word[-2] = 'm'
                sing = "".join(new_word)
                sing = sing[:-1]
                return sing

            if new_word[-1] == 's' and new_word[-2] == 'e' and new_word[-3] == 'õ':
                new_word[-3] = 'ã'
                new_word[-2] = 'o'
                sing = "".join(new_word)
                sing = sing[:-1]
                return  sing

            if new_word[-1] == 's' and new_word[-2] == 'e' and new_word[-3] == 'ã':
                if word == 'mães':
                    word = word[:-1]
                    return word
                else:
                    new_word[-2] = 'o'
                    sing = "".join(new_word)
                    sing = sing[:-1]
                    return sing

            if new_word[-1] == 's' and new_word[-2] == 'i' and new_word[-3] == 'a':
                if word != 'cais' and word != 'mais':
                    new_word[-2] = 'l'
                    sing = "".join(new_word)
                    sing = sing[:-1]
                    return sing

            if new_word[-1] == 's' and new_word[-2] == 'i' and new_word[-3] == 'é':
                new_word[-3] = 'e'
                new_word[-2] = 'l'
                sing = "".join(new_word)
                sing = sing[:-1]
                return sing

            if new_word[-1] == 's' and new_word[-2] == 'i' and new_word[-3] == 'e':
                new_word[-3] = 'e'
                new_word[-2] = 'l'
                sing = "".join(new_word)
                sing = sing[:-1]
                return sing

            if new_word[-1] == 's' and new_word[-2] == 'i' and new_word[-3] == 'ó':
                new_word[-3] = 'o'
                new_word[-2] = 'l'
                sing = "".join(new_word)
                sing = sing[:-1]
                return sing

            if new_word[-1] == 's' and new_word[-2] == 'i':
                if word not in excep:
                    new_word[-1] = 'l'
                    sing = "".join(new_word)
                    return sing

            if new_word[-1] == 's' and new_word[-2] == 'e' and new_word[-3] == 'l':
                word = word[:-2]
                return word

            if new_word[-1] == 's' and new_word[-2] == 'e' and new_word[-3] == 'r':
                word = word[:-2]
                return word

            if new_word[-1] == 's':
                if word not in excep_s:
                    word = word[:-1]

        return word

    def stem(self, word):
        word = self.__plural_reduction(word)

        return word

In [47]:
def preprocess(txt):
    txt = str(txt)
    stemmer = RSLP_S()
    terms = word_tokenize(txt)
    terms = [stemmer.stem(word) for word in terms]
    terms = " ".join(terms)
    return terms

In [48]:
dados1["txtInteiroTeor"] = dados1["txtInteiroTeor"].map(preprocess)

In [49]:
dados1.to_json("rslps/pre_processamento_rslps.json",orient="records")

## Letra mínuscula + remoção de pontuação + remoção de acentuação e remoção de stopword + stemming (RSLP)

In [14]:
dados = pd.read_csv("base_20230428_douglas.csv")

In [15]:
colId = range(len(dados))

In [16]:
dados = dados[["txtNome","txtInteiroTeor"]]
dados1 = dados.assign(id=colId)
dados1 = dados1.rename(columns={"txtInteiroTeor":"contents"})

In [17]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = str(txt)
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))

    stemmer = RSLPStemmer()
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [stemmer.stem(word) for word in terms if word not in stopwords]
        
    return " ".join(terms)

In [18]:
a = time.time()

In [19]:
dados1["contents"] = dados1["contents"].map(preprocess)

In [20]:
d = time.time()

In [21]:
print("Duração: %f" %(d-a))

Duração: 1202.042559


In [26]:
dados1.to_json("rslp_full/pre_processamento_rslp_full.json",orient="records")

## Letra mínuscula + remoção de pontuação + remoção de acentuação e remoção de stopword + stemming (Savoy)

In [30]:
dados = pd.read_csv("base_20230428_douglas.csv")

In [31]:
colId = range(len(dados))

In [32]:
dados = dados[["txtNome","txtInteiroTeor"]]
dados1 = dados.assign(id=colId)
dados1 = dados1.rename(columns={"txtInteiroTeor":"contents"})

In [33]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = str(txt)
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))

    stemmer = Savoy()
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [stemmer.stem(word) for word in terms if word not in stopwords]
        
    return " ".join(terms)

In [34]:
a = time.time()

In [35]:
dados1["contents"] = dados1["contents"].map(preprocess)

In [36]:
d = time.time()

In [37]:
print("Duração: %f" %(d-a))

Duração: 340.090966


In [32]:
dados1.to_json("savoy_full/pre_processamento_savoy_full.json",orient="records")

## Letra mínuscula + remoção de pontuação + remoção de acentuação e remoção de stopword + stemming (RSLP-S)

In [38]:
dados = pd.read_csv("base_20230428_douglas.csv")

In [39]:
colId = range(len(dados))

In [40]:
dados = dados[["txtNome","txtInteiroTeor"]]
dados1 = dados.assign(id=colId)

In [41]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = str(txt)
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))

    stemmer = RSLP_S()
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [stemmer.stem(word) for word in terms if word not in stopwords]
        
    return " ".join(terms)

In [42]:
a = time.time()

In [43]:
dados1["txtInteiroTeor"] = dados1["txtInteiroTeor"].map(preprocess)

In [44]:
d = time.time()

In [45]:
print("Duração: %f" %(d-a))

Duração: 185.940062


In [40]:
dados1.to_json("rslps_full/pre_processamento_rslps_full.json",orient="records")

## Bigram

In [68]:
dados = pd.read_csv("base_20230428_douglas.csv")

In [69]:
colId = range(len(dados))

In [70]:
dados = dados[["txtNome","txtInteiroTeor"]]
dados1 = dados.assign(id=colId)

In [71]:
def preprocess(txt):
    txt = str(txt)
    terms = word_tokenize(txt)
    ngram = []
    
    ngram_2 = list(ngrams(terms, 2))
        
    for w in ngram_2:
        string = w[0] + "#" + w[1]
        ngram.append(string)
    
    return " ".join(ngram)

In [72]:
dados1["txtInteiroTeor"] = dados1["txtInteiroTeor"].map(preprocess)

In [73]:
dados1 = dados1.rename(columns={"txtInteiroTeor":"contents"})

In [74]:
dados1.to_json("bigram/pre_processamento_bigram.json",orient="records")

## Trigram

In [75]:
dados = pd.read_csv("base_20230428_douglas.csv")

In [76]:
colId = range(len(dados))

In [77]:
dados = dados[["txtNome","txtInteiroTeor"]]
dados1 = dados.assign(id=colId)

In [78]:
def preprocess(txt):
    txt = str(txt)
    terms = word_tokenize(txt)
    ngram = []
    
    ngram_3 = list(ngrams(terms, 3))
        
    for w in ngram_3:
        string = w[0] + "#" + w[1] + "#" + w[2]
        ngram.append(string)
    
    return " ".join(ngram)

In [79]:
dados1["txtInteiroTeor"] = dados1["txtInteiroTeor"].map(preprocess)

In [80]:
dados1 = dados1.rename(columns={"txtInteiroTeor":"contents"})

In [81]:
dados1.to_json("trigram/pre_processamento_trigram.json",orient="records")

## Unigram + Bigram

In [82]:
dados = pd.read_csv("base_20230428_douglas.csv")

In [83]:
colId = range(len(dados))

In [84]:
dados = dados[["txtNome","txtInteiroTeor"]]
dados1 = dados.assign(id=colId)

In [85]:
def preprocess(txt):
    txt = str(txt)
    terms = word_tokenize(txt)
    
    ngram = []
    ngram_1 = list(ngrams(terms, 1))
    ngram_2 = list(ngrams(terms, 2))
    for w in ngram_1:
        ngram.append(w[0])
        
    for w in ngram_2:
        string = w[0] + "#" + w[1]
        ngram.append(string)
    
    return " ".join(ngram)

In [86]:
dados1["txtInteiroTeor"] = dados1["txtInteiroTeor"].map(preprocess)

In [87]:
dados1 = dados1.rename(columns={"txtInteiroTeor":"contents"})

In [88]:
dados1.to_json("unigram_bigram/pre_processamento_unigram_bigram.json",orient="records")

# Word n-gram + pré processamento básico

## Letra mínuscula + remoção de pontuação, acentuação e stopword + bigram

In [89]:
dados = pd.read_csv("base_20230428_douglas.csv")

In [90]:
colId = range(len(dados))

In [91]:
dados = dados[["txtNome","txtInteiroTeor"]]
dados1 = dados.assign(id=colId)

In [92]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = str(txt)
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))

    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [word for word in terms if word not in stopwords]
    
    ngram = []
    ngram_2 = list(ngrams(terms, 2))
        
    for w in ngram_2:
        string = w[0] + "#" + w[1]
        ngram.append(string)
    
    return " ".join(ngram)

In [93]:
dados1["txtInteiroTeor"] = dados1["txtInteiroTeor"].map(preprocess)

In [94]:
dados1 = dados1.rename(columns={"txtInteiroTeor":"contents"})

In [95]:
dados1.to_json("bigram_full/pre_processamento_bigram_full.json",orient="records")

## Letra mínuscula + remoção de pontuação, acentuação e stopword + trigram

In [96]:
dados = pd.read_csv("base_20230428_douglas.csv")

In [97]:
colId = range(len(dados))

In [98]:
dados = dados[["txtNome","txtInteiroTeor"]]
dados1 = dados.assign(id=colId)

In [99]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = str(txt)
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))

    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [word for word in terms if word not in stopwords]
    
    ngram = []
    ngram_3 = list(ngrams(terms, 3))
        
    for w in ngram_3:
        string = w[0] + "#" + w[1] + "#" + w[2]
        ngram.append(string)
    
    return " ".join(ngram)

In [100]:
dados1["txtInteiroTeor"] = dados1["txtInteiroTeor"].map(preprocess)

In [101]:
dados1 = dados1.rename(columns={"txtInteiroTeor":"contents"})

In [102]:
dados1.to_json("trigram_full/pre_processamento_trigram_full.json",orient="records")

## Letra mínuscula + remoção de pontuação, acentuação e stopword + unigram + bigram

In [103]:
dados = pd.read_csv("base_20230428_douglas.csv")

In [104]:
colId = range(len(dados))

In [105]:
dados = dados[["txtNome","txtInteiroTeor"]]
dados1 = dados.assign(id=colId)

In [106]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = str(txt)
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))

    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [word for word in terms if word not in stopwords]
    
    ngram = []
    ngram_1 = list(ngrams(terms, 1))
    ngram_2 = list(ngrams(terms, 2))
    for w in ngram_1:
        ngram.append(w[0])
        
    for w in ngram_2:
        string = w[0] + "#" + w[1]
        ngram.append(string)
    
    return " ".join(ngram)

In [107]:
dados1["txtInteiroTeor"] = dados1["txtInteiroTeor"].map(preprocess)

In [108]:
dados1 = dados1.rename(columns={"txtInteiroTeor":"contents"})

In [109]:
dados1.to_json("unigram_bigram_full/pre_processamento_unigram_bigram_full.json",orient="records")

# Word n-gram + pré processamento básico + rslp

## Letra mínuscula + remoção de pontuação, acentuação e stopword + bigram (rslp)

In [ ]:
dados = pd.read_csv("base_20230428_douglas.csv")

In [ ]:
colId = range(len(dados))

In [ ]:
dados = dados[["txtNome","txtInteiroTeor"]]
dados1 = dados.assign(id=colId)

In [ ]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = str(txt)
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))

    stemmer = RSLPStemmer()
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [stemmer.stem(word) for word in terms if word not in stopwords]

    ngram = []
    ngram_2 = list(ngrams(terms, 2))

    for w in ngram_2:
        string = w[0] + "_" + w[1]
        ngram.append(string)

    return " ".join(ngram)

In [ ]:
dados1["txtInteiroTeor"] = dados1["txtInteiroTeor"].map(preprocess)

In [ ]:
dados1 = dados1.rename(columns={"txtInteiroTeor":"contents"})

In [180]:
dados1.to_json("bigram_rslp_full/pre_processamento_bigram__rslp_full.json",orient="records")

## Letra mínuscula + remoção de pontuação, acentuação e stopword + trigram (rslp)

In [181]:
dados = pd.read_csv("base_20230428_douglas.csv")

In [182]:
colId = range(len(dados))

In [183]:
dados = dados[["txtNome","txtInteiroTeor"]]
dados1 = dados.assign(id=colId)

In [184]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = str(txt)
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))

    stemmer = RSLPStemmer()
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [stemmer.stem(word) for word in terms if word not in stopwords]
    
    ngram = []
    ngram_3 = list(ngrams(terms, 3))
        
    for w in ngram_3:
        string = w[0] + "#" + w[1] + "#" + w[2]
        ngram.append(string)
    
    return " ".join(ngram)

In [185]:
dados1["txtInteiroTeor"] = dados1["txtInteiroTeor"].map(preprocess)

In [186]:
dados1 = dados1.rename(columns={"txtInteiroTeor":"contents"})

In [187]:
dados1.to_json("trigram_rslp_full/pre_processamento_trigram_rslp_full.json",orient="records")

## Letra mínuscula + remoção de pontuação, acentuação e stopword + unigram + bigram (rslp)

In [46]:
dados = pd.read_csv("base_20230428_douglas.csv")

In [47]:
colId = range(len(dados))

In [48]:
dados = dados[["txtNome","txtInteiroTeor"]]
dados1 = dados.assign(id=colId)
dados1 = dados1.rename(columns={"txtInteiroTeor":"contents"})

In [49]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = str(txt)
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))

    stemmer = RSLPStemmer()
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [stemmer.stem(word) for word in terms if word not in stopwords]
    
    ngram = []
    ngram_1 = list(ngrams(terms, 1))
    ngram_2 = list(ngrams(terms, 2))
    for w in ngram_1:
        ngram.append(w[0])
        
    for w in ngram_2:
        string = w[0] + "_" + w[1]
        ngram.append(string)
    
    return " ".join(ngram)

In [50]:
a = time.time()

In [51]:
dados1["contents"] = dados1["contents"].map(preprocess)

In [52]:
d = time.time()

In [53]:
print("Duração: %f" %(d-a))

Duração: 1069.067130


In [46]:
dados1.to_json("unigram_bigram_rslp_full/pre_processamento_unigram_bigram_rslp_full1.json",orient="records")

# Word n-gram + pré processamento básico + Savoy

In [ ]:
class Savoy:

    def __removeAllPTAccent(self, old_word):
        word = list(old_word)
        len_word = len(word)-1
        for i in range(len_word, -1, -1):
            if word[i] == 'ä':
                word[i] = 'a'
            if word[i] == 'â':
                word[i] = 'a'
            if word[i] == 'à':
                word[i] = 'a'
            if word[i] == 'á':
                word[i] = 'a'
            if word[i] == 'ã':
                word[i] = 'a'
            if word[i] == 'ê':
                word[i] = 'e'
            if word[i] == 'é':
                word[i] = 'e'
            if word[i] == 'è':
                word[i] = 'e'
            if word[i] == 'ë':
                word[i] = 'e'
            if word[i] == 'ï':
                word[i] = 'i'
            if word[i] == 'î':
                word[i] = 'i'
            if word[i] == 'ì':
                word[i] = 'i'
            if word[i] == 'í':
                word[i] = 'i'
            if word[i] == 'ü':
                word[i] = 'u'
            if word[i] == 'ú':
                word[i] = 'u'
            if word[i] == 'ù':
                word[i] = 'u'
            if word[i] == 'û':
                word[i] = 'u'
            if word[i] == 'ô':
                word[i] = 'o'
            if word[i] == 'ö':
                word[i] = 'o'
            if word[i] == 'ó':
                word[i] = 'o'
            if word[i] == 'ò':
                word[i] = 'o'
            if word[i] == 'õ':
                word[i] = 'o'
            if word[i] == 'ç':
                word[i] = 'c'

        new_word = "".join(word)
        return new_word

    def __finalVowelPortuguese(self, word):
        len_word = len(word)
        if len_word > 3:
            if word[-1] == 'e' or word[-1] == 'a' or word[-1] == 'o':
                word = word[:-1]

        return word

    def __remove_PTsuffix(self, word):
        len_word = len(word)

        if len_word > 3:
            if word[-1] == 's' and word[-2] == 'e' and (word[-3] == 'r' or word[-3] == 's' or word[-3] == 'z' or word[-3] == 'l'):
                word = word[:-2]
                return word
        if len_word > 2:
            if word[-1] == 's' and word[-2] == 'n':
                new_word = list(word)
                new_word[-2] = 'm'
                sing = "".join(new_word)
                sing = sing[:-1]
                return sing

        if len_word > 3:
            if (word[-1] == 's' and word[-2] == 'i') and (word[-3] == 'e' or word[-3] == 'é'):
                new_word = list(word)
                new_word[-3] = 'e'
                new_word[-2] = 'l'
                sing = "".join(new_word)
                sing = sing[:-1]
                return sing

        if len_word > 3:
            if word[-1] == 's' and word[-2] == 'i' and word[-3] == 'a':
                new_word = list(word)
                new_word[-2] = 'l'
                sing = "".join(new_word)
                sing = sing[:-1]
                return sing

        if len_word > 3:
            if word[-1] == 's' and word[-2] == 'i' and word[-3] == 'ó':
                new_word = list(word)
                new_word[-3] = 'o'
                new_word[-2] = 'l'
                sing = "".join(new_word)
                sing = sing[:-1]
                return sing

        if len_word > 3:
            if word[-1] == 's' and word[-2] == 'i':
                new_word = list(word)
                new_word[-1] = 'l'
                sing = "".join(new_word)
                return sing

        if len_word > 2:
            if word[-1] == 's' and word[-2] == 'e' and word[-3] == 'õ':
                new_word = list(word)
                new_word[-3] = 'ã'
                new_word[-2] = 'o'
                sing = "".join(new_word)
                sing = sing[:-1]
                return sing
            if word[-1] == 's' and word[-2] == 'e' and word[-3] == 'ã':
                new_word = list(word)
                new_word[-2] = 'o'
                sing = "".join(new_word)
                sing = sing[:-1]
                return sing

        if len_word > 5:
            if word[-1] == 'e' and word[-2] == 't' and word[-3] == 'n' and word[-4] == 'e' and word[-5] == 'm':
                word = word[:-5]
                return word

        if len_word > 2:
            if word[-1] == 's':
                word = word[:-1]

        return word

    def __normFemininPortuguese(self, word):

        len_word = len(word)

        if len_word < 3 or word[-1] != 'a':
            return word

        if len_word > 6:

            if word[-2] == 'h' and word[-3] == 'n' and word[-4] == 'i':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

            if word[-2] == 'c' and word[-3] == 'a' and word[-4] == 'i':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

            if word[-2] == 'r' and word[-3] == 'i' and word[-4] == 'e':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

        if len_word > 5:
            if word[-2] == 'n' and word[-3] == 'o':
                new_word = list(word)
                new_word[-3] = 'ã'
                new_word[-2] = 'o'
                masc = "".join(new_word)
                masc = masc[:-1]
                return masc

            if word[-2] == 'r' and word[-3] == 'o':
                word = word[:-1]
                return word

            if word[-2] == 's' and word[-3] == 'o':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

            if word[-2] == 's' and word[-3] == 'e':
                new_word = list(word)
                new_word[-3] = 'ê'
                masc = "".join(new_word)
                masc = masc[:-1]
                return masc

            if word[-2] == 'c' and word[-3] == 'i':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

            if word[-2] == 'd' and word[-3] == 'i':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

            if word[-2] == 'd' and word[-3] == 'a':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

            if word[-2] == 'v' and word[-3] == 'i':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

            if word[-2] == 'm' and word[-3] == 'a':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

            if word[-2] == 'n':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

        return word

    def stem(self, word):
        len_word = len(word)
        if len_word > 2:
            word = self.__remove_PTsuffix(word)
            word = self.__normFemininPortuguese(word)
            word = self.__finalVowelPortuguese(word)
            word = self.__removeAllPTAccent(word)

        return word


## Letra mínuscula + remoção de pontuação, acentuação e stopword + bigram (savoy)

In [196]:
dados = pd.read_csv("base_20230428_douglas.csv")

In [197]:
colId = range(len(dados))

In [198]:
dados = dados[["txtNome","txtInteiroTeor"]]
dados1 = dados.assign(id=colId)

In [199]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = str(txt)
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))

    stemmer = Savoy()
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [stemmer.stem(word) for word in terms if word not in stopwords]

    ngram = []
    ngram_2 = list(ngrams(terms, 2))

    for w in ngram_2:
        string = w[0] + "#" + w[1]
        ngram.append(string)

    return " ".join(ngram)

In [200]:
dados1["txtInteiroTeor"] = dados1["txtInteiroTeor"].map(preprocess)

In [201]:
dados1 = dados1.rename(columns={"txtInteiroTeor":"contents"})

In [202]:
dados1.to_json("bigram_savoy_full/pre_processamento_bigram__savoy_full.json",orient="records")

## Letra mínuscula + remoção de pontuação, acentuação e stopword + trigram (savoy)

In [203]:
dados = pd.read_csv("base_20230428_douglas.csv")

In [204]:
colId = range(len(dados))

In [205]:
dados = dados[["txtNome","txtInteiroTeor"]]
dados1 = dados.assign(id=colId)

In [206]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = str(txt)
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))

    stemmer = Savoy()
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [stemmer.stem(word) for word in terms if word not in stopwords]
    
    ngram = []
    ngram_3 = list(ngrams(terms, 3))
        
    for w in ngram_3:
        string = w[0] + "#" + w[1] + "#" + w[2]
        ngram.append(string)
    
    return " ".join(ngram)

In [207]:
dados1["txtInteiroTeor"] = dados1["txtInteiroTeor"].map(preprocess)

In [208]:
dados1 = dados1.rename(columns={"txtInteiroTeor":"contents"})

In [209]:
dados1.to_json("trigram_savoy_full/pre_processamento_trigram_savoy_full.json",orient="records")

## Letra mínuscula + remoção de pontuação, acentuação e stopword + unigram + bigram (savoy)

In [54]:
dados = pd.read_csv("base_20230428_douglas.csv")

In [55]:
colId = range(len(dados))

In [56]:
dados = dados[["txtNome","txtInteiroTeor"]]
dados1 = dados.assign(id=colId)

In [57]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = str(txt)
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))

    stemmer = Savoy()
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [stemmer.stem(word) for word in terms if word not in stopwords]
    
    ngram = []
    ngram_1 = list(ngrams(terms, 1))
    ngram_2 = list(ngrams(terms, 2))
    for w in ngram_1:
        ngram.append(w[0])
        
    for w in ngram_2:
        string = w[0] + "_" + w[1]
        ngram.append(string)
    
    return " ".join(ngram)

In [58]:
a = time.time()

In [59]:
dados1["txtInteiroTeor"] = dados1["txtInteiroTeor"].map(preprocess)

In [60]:
d = time.time()

In [61]:
print("Duração: %f" %(d-a))

Duração: 371.950056


In [53]:
dados1 = dados1.rename(columns={"txtInteiroTeor":"contents"})

In [54]:
dados1.to_json("unigram_bigram_savoy_full/pre_processamento_unigram_bigram_savoy_full.json",orient="records")

# Word n-gram + pré processamento básico + RSLP-S

In [9]:
dados = pd.read_csv("base_20230428_douglas.csv")

In [10]:
colId = range(len(dados))

In [11]:
dados = dados[["txtNome","txtInteiroTeor"]]
dados1 = dados.assign(id=colId)

In [12]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = str(txt)
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))

    stemmer = RSLP_S()
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [stemmer.stem(word) for word in terms if word not in stopwords]

    ngram = []
    ngram_2 = list(ngrams(terms, 2))

    for w in ngram_2:
        string = w[0] + "#" + w[1]
        ngram.append(string)

    return " ".join(ngram)

In [221]:
dados1["txtInteiroTeor"] = dados1["txtInteiroTeor"].map(preprocess)

In [222]:
dados1 = dados1.rename(columns={"txtInteiroTeor":"contents"})

In [223]:
dados1.to_json("bigram_rslps_full/pre_processamento_bigram__rslps_full.json",orient="records")

## Letra mínuscula + remoção de pontuação, acentuação e stopword + trigram (RSLPS)

In [224]:
dados = pd.read_csv("base_20230428_douglas.csv")

In [225]:
colId = range(len(dados))

In [226]:
dados = dados[["txtNome","txtInteiroTeor"]]
dados1 = dados.assign(id=colId)

In [227]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = str(txt)
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))

    stemmer = RSLP_S()
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [stemmer.stem(word) for word in terms if word not in stopwords]
    
    ngram = []
    ngram_3 = list(ngrams(terms, 3))
        
    for w in ngram_3:
        string = w[0] + "#" + w[1] + "#" + w[2]
        ngram.append(string)
    
    return " ".join(ngram)

In [228]:
dados1["txtInteiroTeor"] = dados1["txtInteiroTeor"].map(preprocess)

In [229]:
dados1 = dados1.rename(columns={"txtInteiroTeor":"contents"})

In [230]:
dados1.to_json("trigram_rslps_full/pre_processamento_trigram_rslps_full.json",orient="records")

## Letra mínuscula + remoção de pontuação, acentuação e stopword + unigram + bigram (RSLPS)

In [62]:
dados = pd.read_csv("base_20230428_douglas.csv")

In [63]:
colId = range(len(dados))

In [64]:
dados = dados[["txtNome","txtInteiroTeor"]]
dados1 = dados.assign(id=colId)

In [65]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = str(txt)
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))

    stemmer = RSLP_S()
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [stemmer.stem(word) for word in terms if word not in stopwords]
    
    ngram = []
    ngram_1 = list(ngrams(terms, 1))
    ngram_2 = list(ngrams(terms, 2))
    for w in ngram_1:
        ngram.append(w[0])
        
    for w in ngram_2:
        string = w[0] + "_" + w[1]
        ngram.append(string)
    
    return " ".join(ngram)

In [66]:
a = time.time()

In [67]:
dados1["txtInteiroTeor"] = dados1["txtInteiroTeor"].map(preprocess)

In [68]:
d = time.time()

In [69]:
print("Duração: %f" %(d-a))

Duração: 215.609079


In [60]:
dados1 = dados1.rename(columns={"txtInteiroTeor":"contents"})

In [61]:
dados1.to_json("unigram_bigram_rslps_full/pre_processamento_unigram_bigram_rslps_full.json",orient="records")